In [1]:
# Statistical Hypothesis Testing:
# Need to create hypothesis and test our assertions

# Example:
# This is a fair coin
# Data scientists prefer Ptython to R
# Null Hypothesis H0: default position
# H1: Alternate hypothesis to compare H0 to

# 1. fLIPPING A COIN
# a COIN AND WANT TO TEST IF ITS FAIR
# H0: Assumption that the coin has a prob p of landing heads
# Null Hypothesis - The coin is fair so p = 0.5
# H1: Alternate Hypothesis: p != 0.5

# We will test flipping the coin N times and counting the number of heads X
# Each coin flip is a Bernoulli trial
# so X is a Binomail(n,p) random variable we can approx using the normal distribution:

def normal_approx_to_binomail(n, p):
    
    """Finds mu and sigma corresponding to a Binomail(n,p)"""
    # Mean = probability * N (number of trials) = 0.5 * n
    mu = p * n
    # Standard Dev = sqrt of prob (Null H0) * (1-PROB) (H1:Alt Hypo) * N
    # sqrt(0.5 * (1-0.5) * n)
    sigma = math.sqrt(p * (1-p)*n)
    return mu, sigma
    

SyntaxError: invalid syntax (<ipython-input-1-cc75cf9c3077>, line 20)